In [1]:
import numpy as np
import random

In [2]:
maxPop = 10
maxQ = 8
generation = 0;

In [3]:
def createPopulation():
    population = [[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0], 
                  [0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0],
                  [0,0,0,0,0,0,0,0], [0,0,0,0,0,0,0,0]]
    #random.seed(1)     #Prevent from randomly generated
    for i in range(maxPop):
        for j in range(maxQ):
            population[i][j] = random.randint(1,8)
    return population

In [4]:
def createChromosome(population):
    chromosome = []
    for i in range(maxPop):
        pop_str = ''.join(map(str, population[i]))
        chromosome.insert(i, pop_str)
    return chromosome

In [5]:
def printInformation(obj, str):
    print(str+':')
    for i in range(maxPop):
        if i < maxPop-1:
            print(obj[i], end=', ')
        else:
            print(obj[i])
    print()

In [6]:
def printSolution(obj, str, loc):
    print(str+':')
    for i in range(maxPop):
        if i < maxPop-1:
            if i == loc:
                print("<<{}>>".format(obj[i]), end=', ')
            else:
                print(obj[i], end=', ')
        else:
            if i == loc:
                print("<<{}>>".format(obj[i]))
            else:
                print(obj[i])
    print()

In [7]:
def printChromosomeFitness(chromfit):
    print("Chromosome fitness:")
    for i in range(maxPop):
        (chrom, fit) = extractChromosomeFitness(chromfit[i])
        print("{} Fitness: {}".format(chrom, fit))

In [8]:
def calculateNonAttQueen(chromosome, maxAttQ):
    nonAttQ = [0,0,0,0,0,0,0,0,0,0]
    
    for pop in range(maxPop):
        individual = chromosome[pop]
        #print(individual)
        i = 0
        j = 1
        AttCt = 0
        for x1 in range(maxQ-1):    
            q1 = int(individual[x1]) #row
            #print("({},{})->".format(q1,x1), end='')            
            i = j
            for x2 in range(i,maxQ):
                q2 = int(individual[x2]) #row
                #print("({},{})".format(q2,x2), end='')
                if(q1 == q2):
                    AttCt = AttCt + 1
                if(abs(x2 - x1) == abs(q2 - q1)):
                    AttCt = AttCt + 1
            #print()
            j = j + 1
        nonAttQ[pop] = maxAttQ - AttCt
    return (nonAttQ)

In [9]:
def calculateFitness(nonAttQ):
    fitness = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
    totNonAttQ = 0
    for pop in range(maxPop):
        totNonAttQ = totNonAttQ + nonAttQ[pop]    

    for pop in range(maxPop):
        fitness[pop] = round((nonAttQ[pop]/totNonAttQ)*100, 2)
    return (fitness)

In [10]:
def extractChromosomeFitness(chromf):
    chrs = chromf[0:8]
    fits = chromf[9:14]
    return (chrs, fits)

In [11]:
def mergePopulationFitness(chromosome, fitness):
    chromfit = ["","","","","","","","","",""]
    strfit = ""
    i=0
    for r in chromosome:
        strfit = r + ","+''.join(str(fitness[i]))
        chromfit[i] = strfit
        i = i + 1 
        strfit=""
    return (chromfit)

In [12]:
def selectChromosomes(chromfit):
    newchromosomes = []
    ct = 0
    fittest = 0.0
    newchrom = ""
    for i in range(maxPop):
        (chrom, fit) = extractChromosomeFitness(chromfit[i])
        if float(fit) > 8.0:
            if float(fit) > fittest:
                fittest = float(fit)
                newchrom = chrom
            newchromosomes.insert(ct, chrom)
            ct = ct + 1
    
    for i in range(maxPop-ct):
        newchromosomes.insert(i, newchrom)

    return (newchromosomes)

In [13]:
def getOnePointCrossOver():
    num_mate = int(maxPop/2)
    cr = [-9,-9,-9,-9,-9]
    for i in range(num_mate):
        cr[i] = random.randint(0,maxQ)
    return(cr)

In [14]:
def checkExist(m, mate_order):
    exist = False
    for i in range(maxPop):
        the_member = mate_order[i]
        if the_member != -9:
            if m == the_member:
                exist = True
                break
        else:
            break
    return exist

In [15]:
def pickMate():
    mate_order = np.array([-9,-9,-9,-9,-9,-9,-9,-9, -9, -9], dtype = int)
    ct = 0
    while ct < maxPop:
        m = random.randint(0,maxPop-1)
        exist = checkExist(m, mate_order)
        if exist == False:
            mate_order[ct] = m
            ct = ct + 1
    return (mate_order)

In [16]:
def crossOver(cr, mate, chromosome):
    k=0
    newchromosomes = []
    numOfMate = int(maxPop/2)
    for j in range(numOfMate):
        p1 = mate[k]
        p2 = mate[k+1]
        crp = cr[j]
        parent1 = chromosome[p1]
        parent2 = chromosome[p2]
        substr1 = parent1[crp+1:maxQ]
        substr2 = parent2[crp+1:maxQ]
        newparent1 = parent1[0:crp+1] + substr2
        newparent2 = parent2[0:crp+1] + substr1
        newchromosomes.insert(k, newparent1)
        newchromosomes.insert(k+1, newparent2)
        k = k + 2
    
    return(newchromosomes)

In [17]:
def mutation(chromosome):
    mutated = []
    selected = []
    pm = int((maxPop * maxQ)*0.1)  #Calculate probability of mutation
    for i in range(pm):
        mut = random.randint(0,maxQ-1)  # choose which gen to mutate
        sel = random.randint(0,maxPop-1)  # choose which chromosome to mutate
        selected = chromosome[sel]
        #print("Selected Chromosome {} before mutated at {}: ".format(sel+1, mut+1))
        #print(selected)
        newsel = list(selected)
        newsel[mut] = str(random.randint(0, maxQ-1))
        selected = "".join(newsel)
        chromosome[sel] = selected
        #print("New Chromosome {} after mutated at {}: ".format(sel+1, mut+1))
        #print(chromosome)
    
    return (chromosome)

In [18]:
def main():
    print('"n-Queens Genetic Algorithm"')
    population = createPopulation()
    chromosome = createChromosome(population)
    printInformation(chromosome, "Initial chomosome")
    maxNonAttQ = int((maxQ * (maxQ - 1))/2)
    print("Optimal Non Attacking Queens: {}".format(maxNonAttQ))
    gen = 0
    solution = 0
    while (solution == 0) and (gen < 5000):
        nonAttQ = calculateNonAttQueen(chromosome, maxNonAttQ)
        for i in range(maxPop):
            if nonAttQ[i] == maxNonAttQ:
                solution = nonAttQ[i]
                sol_chromosome = chromosome[i]
                print("Solution Found!")
                solution = 1
                loc = i
                break
        if solution == 0:
            totNonAttQ = np.sum(nonAttQ)
            fitness = calculateFitness(nonAttQ)
            chromFitness = mergePopulationFitness(chromosome, fitness)
            #printChromosomeFitness(chromFitness)
            chromosome = selectChromosomes(chromFitness)
            cr = getOnePointCrossOver()
            mate = pickMate()
            chromosome = crossOver(cr, mate, chromosome)
            chromosome = mutation(chromosome)
            gen = gen + 1
        
    if(solution == 1):
        print("Optimal Solution found in {} generations".format(gen))
        printSolution(chromosome, "Final chromosome", loc)
        printSolution(nonAttQ, "Optimal non attacking queen", loc)
        print("Solution chromosome: {}".format(sol_chromosome))
    else:
        print("No optimal solution found in {} generations...".format(gen))
        printInformation(nonAttQ, "Non optimal solution found")
 
if __name__ == "__main__":
    main()


"n-Queens Genetic Algorithm"
Initial chomosome:
37766778, 65515545, 56358865, 52315578, 31863231, 66745715, 46617827, 81561342, 42823341, 36624663

Optimal Non Attacking Queens: 28
Solution Found!
Optimal Solution found in 3354 generations
Final chromosome:
<<46152073>>, 54665067, 70752075, 06155501, 73355674, 64102562, 50010467, 46605624, 00551374, 43155570

Optimal non attacking queen:
<<28>>, 18, 22, 20, 17, 23, 21, 19, 24, 22

Solution chromosome: 46152073
